In [3]:
import pandas as pd
from modeling import feature_engineering, get_best_params,closest, agg_function,agg_diff_fun, save_object, load_object, model_predict, save_csv
import numpy as np
import reverse_geocoder as revgc
import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier


In [2]:
df_train= pd.read_json('train.json')
df_test= pd.read_json('test.json')
metro= pd.read_csv('MetroStanice.csv')
df_train["training"]=1
df_test["training"]=0

df= pd.concat([df_train,df_test],sort=True).reset_index(drop=True)

mapping_features= pd.read_csv('feature_deduplication.csv')
mydict= dict(zip(mapping_features.original_feature, mapping_features.unique_feature))
#features and photos
df["len_features"] = df["features"].apply(len)
df["len_photos"] = df["photos"].apply(len)
df['features']= df['features'].apply(lambda x: [item.lower() for item in x]) 
df['features_txt']= df['features'].apply(lambda x: ' '.join(x))
df['features']=df['features'].apply(lambda x: [mydict[item] if item in mydict else item for item in x ])
#pocet features
#

#timefromepoch
import datetime
#df['tstamp'] =df['created'].apply(lambda x:  datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').timestamp())
#Datumove
#df["date"]=  df['created'].str[:7]
df['created']=pd.to_datetime(df["created"])
#df['day'] = df['created'].dt.day
#df['days']=(df["created"]-min(df["created"])).apply(lambda x: x.days)
df['dayofweek'] = df['created'].dt.dayofweek
#cylicke
df['hr'] = df['created'].dt.hour
df['mnth'] = df['created'].dt.month
df['nwords_description']= df['description'].apply(lambda x: x.count(' ')+1)
df['price_digits']=df['price'].apply(lambda x: len(str(x)))
df['price'] = np.log(df['price'])

# GEO
coordinates= df[["latitude","longitude"]].values
coordinates_tuple=tuple((x[0],x[1]) for x in coordinates)
adresy=revgc.search(coordinates_tuple)
df=pd.concat([df,pd.DataFrame(adresy)],axis=1)


#label encoding
event_dictionary ={'low' : 1, 'medium' : 2, 'high' : 3} 
df['interest_integer'] =df['interest_level'].map(event_dictionary) 

#interakcie zakladna intuicia
df['bed+bath']=df['bedrooms']+df['bathrooms']
df['price/bed']=df['price']/(df['bedrooms']+1e-16)
df['price/bed+bath']=df['price']/(df['bed+bath']+1e-16)

training='training'
label='interest_integer'

metro=metro.loc[:,['lat','lon']]
metro=metro.to_dict('records')
dfmetro=df.loc[:,['latitude','longitude']]
dfmetro=dfmetro.to_dict('records')
x=[]
for i in dfmetro:
    x.append(closest(metro, i))
df['metro']=x


#drop columns
df=df.drop(columns=['photos','created','listing_id','lat','lon','latitude','longitude'])

Loading formatted geocoded file...


In [ ]:
training='training'
label='interest_integer'

In [13]:
knn_model=KNeighborsClassifier(n_neighbors=50,n_jobs=-1)
steps = [('scaler', StandardScaler()), ('KNN', knn_model)]
knn = Pipeline(steps)
clf=knn
agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
pca_list=[1,2,3,4,5]
num_features=50
df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf=clf)
save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_knn_thesis')

2020-05-30 12:14:15.916308


  7%|▋         | 1/14 [01:04<13:52, 64.05s/it]


KeyboardInterrupt: 

In [6]:
# parameters=pd.read_csv('Pipeline_KNN1_grid_results.csv')
# best_params=get_best_params(parameters,'score')
# best_params
# agg_dict={agg_function: ['mean','median','sum','std'],agg_diff_fun: ['mean', 'median']}
# training='training'
# label='interest_integer'
# pca_list=[1,2,3,4,5]
# num_features=50
# knn_model=KNeighborsClassifier(n_jobs=-1,**best_params)
# steps = [('scaler', StandardScaler()), ('KNN', knn_model)]
# knn = Pipeline(steps)
# clf=knn
# df_out,columns,good_encoders,dp_table,df_tokenizer=feature_engineering(df,training,label,num_features,agg_dict,pca_list,clf =clf)
# save_object([df_out,columns,good_encoders,dp_table,df_tokenizer],clf.__class__.__name__+'_thesis_best_grid_params_extra')